In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import ROOT
from scipy.special import erf
from scipy.integrate import quad
from scipy import signal
from scipy.optimize import curve_fit
import platform
from multiprocess import Pool, Manager, Array
from tqdm.notebook import tqdm
import gc
import sys
print(platform.python_version())

In [ ]:
dosave=False
plt.rcParams.update({'font.size': 14})

In [ ]:
# Change Me!

# energy scale [MeV]
muonE = 1_000 

In [ ]:
# constants -- units MeV, cm, s
# MeV * cm
# hbarc = 1.973269804 * 1e-11
# cm / s
# c = 2.998 * 1e10

Mmuon = 105.6
Melec = 0.5110
Relec = 2.817940 * 1e-13

# motion -- beta
beta = np.sqrt(1 - (Mmuon * Mmuon) / (muonE * muonE))
# motion -- gamma
gamma = muonE / Mmuon
# motion -- (gamma * beta)^2
gammaB2 = gamma*gamma*beta*beta

# electron ionization energy [MeV]
I0 = 188.0 * 1e-6

# maximum energy transfer [Mev]
# maxE = 2 * Melec * gammaB2
maxE = 2 * Melec * gammaB2 / \
    ( 1 + 2 * gamma * Melec / Mmuon + (Melec/Mmuon)**2)

# get the density of argon 
# density -- g / mL
LAr_density_gmL = 1.396
# molar mass -- g / mol
Ar_molar_mass = 39.9623
# avogadro number
mole = 6.0221409*1e23
# charge number / atomic mass (Z / A)
# 99.6% of LAr is Ar40
Ar_ZA = 18. / Ar_molar_mass

# electron number density (N / cm^3)
density = Ar_ZA * mole * LAr_density_gmL

# wirep (cm)
wirep = 0.3

# MeV cm^2 / mol
K = 4*np.pi*mole*Relec**2*Melec # 0.307075

# outputdir = "/home/grayputnam/Work/Summer2020/July13BetheBlochBarkas/"
zeta = (K/2.)*Ar_ZA*(1./beta**2) * LAr_density_gmL / maxE

def f_kappa(thickness):
    return zeta * thickness

mean_dEdx = LAr_density_gmL * K * Ar_ZA * (1 / (beta * beta)) * (\
        (1./2)*np.log( maxE*2 * gammaB2*Melec / (I0**2)) \
        - beta * beta)

def f_mpv(thickness):
    kappa = f_kappa(thickness)*maxE
    j=0.200
    return (kappa/thickness)*\
    (np.log(2 * Melec * gammaB2 / I0) + np.log(kappa/I0) + j - beta*beta)

def smeared_dep(x0, w, sigma):
    if sigma < 1e-4:
        return 1*((x0 > -w/2) & (x0 <= w/2))
    return (1./2)*(erf((w/2+x0)/(np.sqrt(2)*sigma)) +\
                   erf((w/2-x0)/(np.sqrt(2)*sigma)))

def f_thickness(sigma, a=wirep):
    return a*np.exp(quad(lambda x: -smeared_dep(x, a, sigma) * np.log(smeared_dep(x, a, sigma))/a, 
                       -(a/2) - 5*sigma, (a/2) + 5*sigma)[0])

Dtransverse = 5.85e-3 # cm^2/ms
def smearing(driftT):
    return np.sqrt(2*Dtransverse*driftT)

In [ ]:
# ArgoNeuT
f_mpv(0.4), f_mpv(f_thickness(smearing(0.295), 0.4))

In [ ]:
# MicroBooNE
f_mpv(0.3), f_mpv(f_thickness(smearing(2.33), 0.3))

In [ ]:
# ICARUS
f_mpv(0.3), f_mpv(f_thickness(smearing(0.96), 0.3))

In [ ]:
# SBND
f_mpv(0.3), f_mpv(f_thickness(smearing(1.28), 0.3))

In [ ]:
# DUNE-FD
f_mpv(0.47), f_mpv(f_thickness(smearing(2.2), 0.47))

In [ ]:
# Plot thickness dependance
sigmas = np.linspace(0, 2, 21)
thicks = [f_thickness(s, 1) for s in sigmas]

plt.plot(sigmas, thicks, label="Step Function(a)\n $\\circledast$ Gaussian($\\sigma$)")
plt.plot(sigmas, np.sqrt(2*np.pi*np.e)*sigmas, linestyle=":", color="black", label="Large $\\sigma$ Limit")
# plt.gca().set_xticklabels(["%.1fa" % l for l in plt.xticks()[0]])
plt.xlabel("Gaussian Width ($\\sigma$) / Wire Pitch ($a$)")
plt.ylabel("Thickness ($\\mathscr{t}$) / Wire Pitch ($a$)")

plt.legend()
plt.tight_layout()

if dosave: plt.savefig(savedir + "step_gaussian_thickness.png")

In [ ]:
def xsec(e):
    # front constant of xsec
    const = (2 * np.pi  * Relec * Relec * Melec) / (beta * beta)
    A = 1
    C = -beta * beta / maxE
    ret = const * (A / (e * e) + C / e)
    ret[e > maxE] = 0
    return ret

def xsec_integral(e): 
    if e > maxE:
        return xsec_integral(maxE, sign, correction)
    # front constant of xsec
    const = (2 * np.pi  * Relec * Relec * Melec) / (beta * beta)
    A = 1
    C = -beta * beta / maxE
    ret = const * (A * (1./I0 - 1./e) + \
            C * np.log(e/I0))
    return ret
total_xsec = xsec_integral(maxE)

In [ ]:
def selfconvolve(a):
    a_fft = np.fft.rfft(a)
    a_fft_2 = a_fft * a_fft
    del a_fft
    return np.fft.irfft(a_fft_2)

def doconvolve(a, b):
    a_fft = np.fft.rfft(a)
    b_fft = np.fft.rfft(b)
    ab_fft = a_fft*b_fft
    del a_fft
    del b_fft
    return np.fft.irfft(ab_fft)

In [ ]:
n_sample = 50_000_000
n_sample = 100_000_000

pitch = 1e-2

# this_maxE = np.minimum(maxE/3, 10)
this_maxE = 10
this_maxE = 20

energies = np.linspace(0, this_maxE, n_sample, endpoint=False)

dx0 = pitch
i = 0
while dx0 > 1e-8:
    i += 1
    dx0 = dx0 / 2
    
dE = this_maxE / n_sample

f0 = np.zeros((n_sample,))
f0[energies >= I0] = dx0 * xsec(energies[energies >= I0]) * density 
f0[0] += (1 - density * total_xsec * dx0) / dE

In [ ]:
dx = dx0
f = f0
i_conv = 0
fs = []
pitches = []
while dx < pitch:
    # if dx >= 0.005:
    #     fs.append(f)
    #     pitches.append(dx)
        
    # determine how many samples for the next run
    # this_maxE = n_sample * dE
    # n_sample = max(n_sample, int(maxE * dx / dE))
    # print(n_sample)
    f = selfconvolve(f) * dE
    dx = dx * 2
    f = f / np.sum(f*dE)
    i_conv += 1
    print(i_conv, dx)
    
    gc.collect()
    
# fs.append(f)
# pitches.append(dx)

In [ ]:
mean_dEdx = LAr_density_gmL * K * Ar_ZA * (1 / (beta * beta)) * (\
        (1./2)*np.log( maxE*2 * gammaB2*Melec / (I0**2)) \
        - beta * beta)

delta_dE = LAr_density_gmL * K * Ar_ZA  * (1. / (beta * beta))\
    * (np.log(2*Melec * gammaB2 / I0) / 2 - beta*beta/2)

In [ ]:
downsample_plt = 20

In [ ]:
plt.plot((energies / dx + delta_dE)[::downsample_plt], f[::downsample_plt])
plt.xlim(0,5.)
plt.xlabel("dE/dx [MeV/cm]")
plt.ylabel("Relative Probability")
plt.axvline(f_mpv(pitch), color="r")
plt.text(f_mpv(pitch), 0.5,'M.P.V.', fontsize=14, rotation=90, color="r")
plt.axvline(mean_dEdx, color="r", linestyle=":")
plt.text(mean_dEdx, 1., 'Mean', fontsize=14, rotation=90, color="r")

plt.title("Muon Energy Loss Landau: E=%.0fMeV dx=%.0fcm" % (muonE, pitch))
plt.tight_layout()

if dosave: plt.savefig(savedir + "muon_eloss_p%.0f.png" % pitch)

In [ ]:
downsample = 1


In [ ]:
# fdx = f[::downsample]
fdx = np.mean(f.reshape(-1, downsample), axis=1)

dEw = dE*downsample

fdx = fdx / np.sum(fdx*dEw)
Nsample = 1000
xs = np.linspace(-pitch*(Nsample/2), pitch*(Nsample/2), Nsample+1)
xcenter = (xs[1:] + xs[:-1]) / 2.

In [ ]:
def weighted_f(xs, wirep, smearing):
    fw = np.zeros(len(fdx))
    fw[0] = 1. / dEw
    ws = smeared_dep(xs, wirep, smearing)
    for w in tqdm(ws[ws >= 1. / (n_sample / downsample)], total=len(ws[ws >= 1. / (n_sample / downsample)])):
        f_x = np.zeros(len(fdx))
        if w < 1. / (n_sample / downsample):
            continue
        else:
            nskip = int(1. / w)
            toset = np.unique(np.floor(np.arange((n_sample // downsample))/w).astype(int)) // nskip
            toset = toset[toset < ((n_sample // downsample) // nskip)]
            nset = len(toset)
            f_x[:nset] = np.sum(fdx[:len(fdx)-(len(fdx) % nskip)].reshape(-1, nskip), axis=1)[toset]
            f_x = f_x / np.sum(f_x*dEw)

            fw = doconvolve(fw, f_x)[:len(fdx)] * dEw
            fw = fw / np.sum(fw*dEw)
    return fw

In [ ]:
wirep_chk = 1.
f_chk = weighted_f(xcenter, wirep_chk, 0)

In [ ]:
def ROOTVavilov(Es, length):
    pdf = ROOT.Math.VavilovAccurate(f_kappa(length), beta**2)
    xs = (Es - mean_dEdx*length) / (f_kappa(length)*maxE) +\
        -np.log(f_kappa(length)) + np.euler_gamma - 1 - beta**2 
    return np.array([pdf.Pdf(x) for x in xs])

In [ ]:
Renergies = (energies + delta_dE*wirep_chk)[::downsample*downsample_plt]
Renergies_all = Renergies
Renergies = Renergies[Renergies < 8]
Rpdfs = ROOTVavilov(Renergies, wirep_chk)
norm = (np.sum(f_chk[::downsample_plt][Renergies_all < 8])) / (np.sum(Rpdfs))
Rpdfs = Rpdfs * norm

In [ ]:
fig = plt.figure()

gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1], sharex = ax1)

scale = 1. / np.max(f_chk[::downsample_plt][Renergies_all < 8])

ax1.plot(Renergies/wirep_chk, f_chk[::downsample_plt][Renergies_all < 8]*scale, label="Paper Distribution")
ax1.plot(Renergies/wirep_chk, Rpdfs*scale, color="black", linestyle=":", label="ROOT VavilovAccurate")
ax1.set_xlim([1, 5])
# ax2.set_ylim([0.9875, 1.0125])
ax2.set_ylim([0.99775, 1.00225])
ax1.legend()

ax2.plot(Renergies/wirep_chk, Rpdfs / f_chk[::downsample_plt][Renergies_all < 8], color="black")

ax2.set_xlabel("dE/dx [MeV/cm]")
ax1.set_ylabel("Arb. Probability")

ax1.set_title("Muon Energy Loss: E=%.0fMeV dx=%.2fcm" % (muonE, wirep_chk))
ax1.get_shared_x_axes().join(ax1, ax2)
fig.subplots_adjust(hspace=.0)

ax2.axhline(1, color="red", linestyle="--")
ax2.set_ylabel("ROOT/Paper")

yticks = ax2.yaxis.get_major_ticks()
yticks[-1].label1.set_visible(False)
plt.setp(ax1.get_xticklabels(), visible=False)

ax1.grid(True)
ax2.grid(True)
ax1.axvline(f_mpv(wirep_chk), color="r")
ax1.text(f_mpv(wirep_chk)+0.025, 0.3,'M.P.V.', fontsize=14, rotation=90, color="r")
ax2.axvline(f_mpv(wirep_chk), color="r")

if dosave: plt.savefig(savedir + "muon_eloss_ROOTcomp_E%.0fMeV.png" % muonE)

In [ ]:
print(f_mpv(wirep_chk), (Renergies / wirep_chk)[np.argmax(f_chk[::downsample_plt])], (Renergies / wirep_chk)[np.argmax(Rpdfs)])

In [ ]:
smearings = np.linspace(0, 0.06, 3)
wirep = 0.3

# smearings = [0.03]
fws = [weighted_f(xcenter, wirep, s) for s in smearings]

In [ ]:
dEdxs = (energies/wirep + delta_dE)[::downsample]

In [ ]:
max_dEdxs = [dEdxs[np.argmax(fw)] for fw in fws]

In [ ]:
max_dEdxs #+ 0.004

In [ ]:
[f_mpv(f_thickness(s, wirep)) for s in smearings]

In [ ]:
plt.plot(smearings, max_dEdxs)
plt.plot(smearings, [f_mpv(f_thickness(s, wirep)) for s in smearings])

In [ ]:
plt.plot(dEdxs, fws[0])
print(smearings[0], dEdxs[np.argmax(fws[0])])

ind = 0
plt.plot(dEdxs, fws[ind])
print(smearings[ind], dEdxs[np.argmax(fws[ind])])

plt.xlim([0, 5])